# Champions general statistics

Get the saved games data

In [1]:
import json, os
gamesRawData = []
for i in os.listdir("./games"):
    if i.endswith(".json"):
        with open("./games/"+i,"r") as f:
            gamesRawData.append(json.loads(f.read()))

Initialize the champion stats data structure.

In [2]:
import requests

response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.19.1/data/en_US/champion.json')
champonRawData = json.loads(response.text)

championsStats = {}
for key,champion in champonRawData['data'].items():
    championsStats[int(champion['key'])] = {
                    "champion":champion['name'],
                    "games":0,
                    "gamesPlayed":0,
                    "wins":0,
                    "teamKills":0,
                    "kills":0,
                    "deaths":0,
                    "assists":0,
                    "bans":0
                }


Parse all the games to get statistics by champion

In [3]:
for game in gamesRawData:
    #Count the champion bans
    for t in game["teams"]:
        for b in t["bans"]:
            championsStats[b["championId"]]["bans"] += 1
            championsStats[b["championId"]]["games"]+=1
    
    #Count the kills of the team to later get the kill participation
    teamKills = {100:0,200:0}
    for p in game["participants"]:
        teamKills[p["teamId"]] += p["stats"]["kills"]
    
    #Count the wins and basic stats like kills, deaths and assits
    for p in game["participants"]:
        championsStats[p["championId"]]["games"]+=1
        championsStats[p["championId"]]["gamesPlayed"]+=1
        if p["stats"]["win"]:
            championsStats[p["championId"]]["wins"] += 1
        
        championsStats[p["championId"]]["teamKills"] += teamKills[p["teamId"]]
        championsStats[p["championId"]]["kills"] += p["stats"]["kills"]
        championsStats[p["championId"]]["deaths"] += p["stats"]["deaths"]
        championsStats[p["championId"]]["assists"] += p["stats"]["assists"]

We make a difference between games and gamesPlayed where games counts every game where the champion has been either played or banned.

Use pandas Dataframe to manipulate the data

In [4]:
import pandas as pd, numpy as np
df = pd.DataFrame(championsStats).T

#Get the winrate for each champion
#wr is the raw winrate
df["wr"] = 0
df.loc[df["gamesPlayed"]>0,"wr"] = df.loc[df["gamesPlayed"]>0]["wins"]/df.loc[df["gamesPlayed"]>0]["gamesPlayed"]
#winrate is the beautified winrate
df["winrate"] = (df["wr"] * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

#Get the KDA
df["KDA"] = 0
df.loc[df["deaths"]>0,"KDA"] = ((df.loc[df["deaths"]>0]["kills"] + df.loc[df["deaths"]>0]["assists"]) / df.loc[df["deaths"]>0]["deaths"])
#Handling the case when KDA is infinite (deaths=0)
df.loc[df["deaths"]==0,"KDA"]=(df.loc[df["deaths"]==0]["kills"] + df.loc[df["deaths"]==0]["assists"])
#Beautify KDA
df["KDA"] = df["KDA"].map('{:,.2f}'.format)

#Get kill participation and beautify it
df["KP"] = 0
df.loc[df["teamKills"]>0,"KP"] = (((df.loc[df["teamKills"]>0]["kills"] + df.loc[df["teamKills"]>0]["assists"]) / df.loc[df["teamKills"]>0]["teamKills"]) * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

Resulting Dataframe

In [5]:
df

,assists,bans,champion,deaths,games,gamesPlayed,kills,teamKills,wins,wr,winrate,KDA,KP
1,0,0,Annie,0,0,0,0,0,0,0,0.00%,0.00,0
2,115,28,Olaf,52,52,24,68,262,9,0.375,37.50%,3.52,69.85%
3,100,22,Galio,40,43,21,48,220,11,0.52381,52.38%,3.70,67.27%
4,0,0,Twisted Fate,0,0,0,0,0,0,0,0.00%,0.00,0
5,97,15,Xin Zhao,44,33,18,45,220,11,0.611111,61.11%,3.23,64.55%
6,164,51,Urgot,90,90,39,120,476,23,0.589744,58.97%,3.16,59.66%
7,78,36,LeBlanc,40,54,18,59,231,10,0.555556,55.56%,3.42,59.31%
8,0,0,Vladimir,0,0,0,0,0,0,0,0.00%,0.00,0
9,0,0,Fiddlesticks,0,0,0,0,0,0,0,0.00%,0.00,0
10,0,0,Kayle,0,0,0,0,0,0,0,0.00%,0.00,0


Exporting the data to csv

In [6]:
df.index.name='championId'
df.to_csv("champion.csv")

Some manipulation examples

In [7]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["wr","gamesPlayed"], ascending=False) #Sorting champions by winrate and games played
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
championId,,,,,,,,,,,,
27,Singed,7,3,14,7.00,35,60.00%,2,2,4,2,100.00%
41,Gangplank,5,3,17,7.33,35,62.86%,2,2,3,1,100.00%
43,Karma,3,2,16,9.50,24,79.17%,2,2,3,1,100.00%
154,Zac,3,1,21,24.00,31,77.42%,2,2,3,1,100.00%
24,Jax,2,1,4,6.00,12,50.00%,1,1,2,1,100.00%
34,Anivia,3,0,12,15.00,18,83.33%,1,1,5,4,100.00%
120,Hecarim,5,2,3,4.00,13,61.54%,1,1,3,2,100.00%
122,Darius,1,1,5,6.00,11,54.55%,1,1,5,4,100.00%
157,Yasuo,1,1,12,13.00,16,81.25%,1,1,6,5,100.00%


In [8]:
#Getting percentage of champions either played or banned in Worlds 2018
df[df["games"] > 0].shape[0] / df.shape[0]

0.6312056737588653

In [9]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["games"], ascending=False) #Sorting champions by games (played and banned)
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
championId,,,,,,,,,,,,
266,Aatrox,64,64,114,2.78,323,55.11%,13,28,92,64,46.43%
6,Urgot,120,90,164,3.16,476,59.66%,23,39,90,51,58.97%
12,Alistar,26,91,270,3.25,417,70.98%,23,38,87,49,60.53%
145,Kai'Sa,264,97,230,5.09,706,69.97%,34,57,76,19,59.65%
223,Tahm Kench,16,56,141,2.80,250,62.80%,10,24,69,45,41.67%
84,Akali,67,34,63,3.82,191,68.06%,8,16,68,52,50.00%
497,Rakan,27,78,244,3.47,344,78.78%,15,32,68,36,46.88%
39,Irelia,79,61,99,2.92,284,62.68%,14,20,65,45,70.00%
79,Gragas,54,105,226,2.67,429,65.27%,16,38,59,21,42.11%
